In [11]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver    
import time
import pandas as pd
import re
import collections
import nltk
from nltk import word_tokenize, Text, FreqDist
from nltk.corpus import wordnet as wn
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
wnl = nltk.WordNetLemmatizer()
import pathlib
from newspaper import Article
from urllib.parse import urlparse
# importing itertools for accumulate()
import itertools
# importing functools for reduce()
import functools
from tqdm.notebook import tqdm_notebook
import json
import praw
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import time
from tqdm import tqdm
import gc
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [89]:
client_id = "sAagBNvoj_godvd7yHezMg" 
client_secret = "cifjzGjeWkDhX1XA2zuTGW-X_vPevg"
user_agent = "testing_api"
username = "xiaochen93"
password = "Xiaopangzi93!"

# Praw Reddit

In [102]:
try:
    reddit = praw.Reddit(client_id = client_id,
                         client_secret = client_secret,
                         user_agent = user_agent,
                         username = username,
                         password = password
                        )
    print(reddit.user.me())
except Exception as e:
    print(e)

xiaochen93


In [105]:
subreddit = reddit.subreddit('singapore')
top = subreddit.top(limit=10)

for submission in top:
    print(dir(submission))
    print(submission.title)
    break

['STR_FIELD', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chunk', '_comments_by_id', '_fetch', '_fetch_data', '_fetch_info', '_fetched', '_kind', '_reddit', '_reset_attributes', '_safely_add_arguments', '_url_parts', '_vote', 'all_awardings', 'allow_live_comments', 'approved_at_utc', 'approved_by', 'archived', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'award', 'awarders', 'banned_at_utc', 'banned_by', 'can_gild', 'can_mod_post', 'category', 'clear_

In [123]:
for comment in submission.comments:
    print(comment.body)
    print(dir(comment))
    break

"beats" to study to indeed
['MISSING_COMMENT_MESSAGE', 'STR_FIELD', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_extract_submission_id', '_fetch', '_fetch_data', '_fetch_info', '_fetched', '_kind', '_reddit', '_replies', '_reset_attributes', '_safely_add_arguments', '_submission', '_url_parts', '_vote', 'all_awardings', 'approved_at_utc', 'approved_by', 'archived', 'associated_award', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'award', 'awarders', 'b

# Selenium Web Crawler

In [5]:
from urllib.parse import urlparse

domain = urlparse('http://www.example.test/foo/bar').netloc
print(domain) # --> www.example.test

www.example.test


In [82]:
url = "https://old.reddit.com/r/singapore/top/"

out = []

def RedditSearch(url,label):
    out = []
    #initalise crawler option(s)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    options.add_argument('headless')
    ROOT_DIR = pathlib.Path().absolute()
    print(ROOT_DIR)
    driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)
    driver.set_window_size(1120, 1000)  
    driver.get(url)
    time.sleep(1)
    
    #1. click to the time series
    xpath_dropdown = "//div[contains(@class,'dropdown lightdrop')]"
    xpath_choices = "//DIV[contains(@class,'drop-choices')]//A[contains(@class,'choice')]"
    #1. allocate the dropdown menu and click the timeseries
    timeseries = getDropdownChoices(driver,xpath_dropdown, xpath_choices)
    timeseries = [choice for choice in timeseries if choice.text == label]
    
    if not timeseries == []:
        timeseries[0].click()
        time.sleep(2)
    else:
        pass
    
    #3. loop and retrieve items on the page.
    searching = True
    noOfDocs = 0
    while searching:
        #2. load main page and pick relative items
        xpath_main_page_items = "//div[contains(@class,'thing id-t3')  and .//SPAN/@title='News']"
        
        items = getTableItems(driver, xpath_main_page_items)
    
        for item in items:
            # get the post id
            try:
                post_id = item.get_property("id")
            except NoSuchElementException:
                print('-- post id not found')
                post_id = ""
            # get the news title
            try:
                title = item.find_element("xpath",".//descendant-or-self::A[contains(@class,'title may-blank')]").text
            except NoSuchElementException:
                title = ""
            # get the news URL
            try:
                title_url = item.find_element("xpath",".//descendant-or-self::A[contains(@class,'title may-blank')]").get_property('href')
            except NoSuchElementException:
                title_url = ""
            # get the datetime
            try:
                datetime  = item.find_element("xpath", ".//descendant-or-self::p[contains(@class, 'tagline')]/time").get_attribute('datetime')
            except NoSuchElementException:
                datetime = ""
            # get the domain name
            try:
                domain = item.find_element("xpath", ".//descendant-or-self::SPAN[contains(@class, 'domain')]").text
            except NoSuchElementException:
                domain = ""

            domain = urlparse(title_url).netloc
            # get the news score
            try:
                scores = item.find_element("xpath", ".//descendant-or-self::div[contains(@class, 'score unvoted')]").text
            except NoSuchElementException:
                scores = ""
            # get the no of comments
            try:
                no_of_cmts = item.find_element("xpath", ".//descendant-or-self::li[contains(@class,'first')]/a").text.split()[0]
            except NoSuchElementException:
                no_of_cmts = "0"
            # get the comment URL
            try:
                cmt_url = item.find_element("xpath", ".//descendant-or-self::li[contains(@class,'first')]/a").get_property('href')
            except NosuchElementException:
                cmt_url = ""
            # get the article content
            content = getNewsContentByArticle(Article, title_url)

            if content == "" or "reddit" in title_url:
                out_dict = getNewsContentByGoogle(title)
                content = out_dict['content']
                domain = out_dict['domain']

            xpath_comment_items = "//div[contains(@class, 'sitetable nestedlisting')]/div[contains(@class, 'comment')]//div[contains(@class, 'md')]/p"
            
            # direct to the comment page and forward control to the new tab
            driver.execute_script("window.open('{}','new window')".format(cmt_url))

            allWindows = driver.window_handles

            # give control to the new page
            driver.switch_to.window(allWindows[len(allWindows)-1])

            #print('\n-- Windows:', allWindows)

            time.sleep(2)

            try:
                comments = driver.find_elements("xpath",xpath_comment_items)
            except NosuchElementException:
                comments = []

            comments = [each.text for each in comments if not each.text == ""]

            #print('/n-- ',post_id, title, title_url, datetime, scores,no_of_cmts, content, cmt_url, comments) 

            driver.switch_to.window(allWindows[0])
            
            time.sleep(1)
            
            one_instance = {
                'p_id': post_id,
                'title': title,
                'p_url':title_url,
                'domain':domain,
                'datetime': datetime,
                'scores': scores,
                'no_of_cmts': no_of_cmts,
                'article': content,
                'c_url': cmt_url,
                'comments': comments
            }
            out.append(one_instance)
            
            noOfDocs = noOfDocs + 1
            print('\n-- {} no of records have collected.'.format(noOfDocs))
            
        # click next to go
        xpath_next = "//div[contains(@class, 'nav-buttons')]//a[contains(text(), 'next')]"
        # Wait for initialize, in seconds
        try:
            clickToGo(driver, xpath_next)
        except:
            break

    return {"data":out}

'''

Input : Xpath for the dropdown list 
Output: Xpath for the choices in the dropdown

'''
def getNewsContentByArticle(Article, url):
    # get content 
    try:
        content = Article(url)
        content.download()
        content.parse()
        content = content.text
    except Exception as e:
        print(e)
        content = ""
    return content

def getNewsContentByGoogle(title):
    
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        
    channels = ["channelnewsasia", "straitstime","todayonline"]
    
    # to search
    query = "news:" + title
 
    content, domain, url = "", "", ""
    
    for j in search(query, tld="co.in", num=10, stop=10, pause=2):
        url = j
        
        domain = urlparse(j).netloc
        
        isIn = [True for c in channels if c in domain]
        
        content = getNewsContentByArticle(Article,j)
        
        if not isIn == []:
            break
    
    return {'content':content, 'domain':domain,'url':url}
        

def getTableItems(driver, xpath_items):
    try:
        items = driver.find_elements("xpath", xpath_items)
    except:
        items = []
        pass
    return items

def getDropdownChoices(driver, xpath_dropdown, xpath_choices):
    # Wait for initialize, in seconds
    wait = WebDriverWait(driver, 10)

    dropdown = wait.until(EC.visibility_of_element_located((By.XPATH, xpath_dropdown)))
    
    time.sleep(1)
    
    dropdown.click()
    
    #2. find the choices on the list
    try:
        dropdown = driver.find_elements("xpath",xpath_choices)
    except ElementClickInterceptedException:
        print('\n-- No dropdown list found --')
        pass
    
    return dropdown

def getChildElement(node, xpath):
    
    xpath = ".//descendant-or-self::" + xpath
    
    try:
        child_node = node.find_element("xpath", xpath)
    except NoSuchElementException:
        print("\n-- Unable to find the child element")
        raise
        
    return child_node


def clickToGo(driver, xpath):
    try:
        button = driver.find_element("xpath",xpath)
        button.click()
        time.sleep(1)
    except:
        print('\n-- No element {} found --'.format(xpath))
        raise
 


In [83]:
items = RedditSearch(url,"past month")

C:\Users\liux5\Documents\Project 3 - DSTA


<ipython-input-82-3b1810461514>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)



-- 1 no of records have collected.

-- 2 no of records have collected.

-- 3 no of records have collected.

-- 4 no of records have collected.

-- 5 no of records have collected.

-- 6 no of records have collected.

-- 7 no of records have collected.

-- 8 no of records have collected.

-- 9 no of records have collected.

-- 10 no of records have collected.

-- 11 no of records have collected.

-- 12 no of records have collected.

-- 13 no of records have collected.

-- 14 no of records have collected.

-- 15 no of records have collected.

-- 16 no of records have collected.

-- 17 no of records have collected.
Article `download()` failed with HTTPSConnectionPool(host='www.reddit.com', port=443): Read timed out. on URL https://www.reddit.com/r/singapore/comments/ttb2hr/all_clear/

-- 18 no of records have collected.

-- 19 no of records have collected.

-- 20 no of records have collected.

-- 21 no of records have collected.

-- 22 no of records have collected.

-- 23 no of records ha

C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "



-- 58 no of records have collected.

-- 59 no of records have collected.

-- 60 no of records have collected.

-- 61 no of records have collected.

-- 62 no of records have collected.

-- 63 no of records have collected.

-- 64 no of records have collected.

-- 65 no of records have collected.

-- 66 no of records have collected.

-- 67 no of records have collected.

-- 68 no of records have collected.

-- 69 no of records have collected.

-- 70 no of records have collected.

-- 71 no of records have collected.

-- 72 no of records have collected.

-- 73 no of records have collected.

-- 74 no of records have collected.

-- 75 no of records have collected.

-- 76 no of records have collected.

-- 77 no of records have collected.

-- 78 no of records have collected.

-- 79 no of records have collected.

-- 80 no of records have collected.

-- 81 no of records have collected.

-- 82 no of records have collected.

-- 83 no of records have collected.

-- 84 no of records have collected.



In [84]:
with open('data/reddit-03.json', 'w', encoding='utf-8') as output_file:
    json.dump(items , output_file ,indent = 2)

# testing functions

In [38]:
HEADERS = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36', #Google Chrome user agent header
        'hl'        :  'en'
    }

s = requests.Session()

s.headers.update(HEADERS)

'''
scrape article content through css selector:
    steps: download page -parse xml - 

'''

def getNewsByCSS(article_url):
    
    article, news_title, category, date, = '', '', '', ''

    try:
        r = s.get(article_url)
        
        soup = BeautifulSoup(r.text, 'lxml')
    except Exception as e:
        print('\n--',e,' for the link:',article_url)
    
    #print(soup)
    try:
        news_title = soup.select('h1[class*="h1--page-title"]')[0].get_text().strip()
    except Exception as e:
        print('\n--',e,' for title')
        
    try:
        category = soup.select('p[class*="content-detail__category"]')[0].get_text().strip()
    except Exception as e:
        print('\n--',e,' for category') 
    
    try:    
        date = soup.select('div[class*="article-publish"]')[0].get_text().strip()
    except Exception as e:
        print('\n--',e,' for date')
    
    try:    
        #get the article text    
        for p_div in soup.select('div[class="text"]>div[class*="text-long"]'):
            for paragraph in p_div.find_all('p'):
                if paragraph.get_text().find('\u00A0') != -1:
                    print(paragraph.get_text())
                    continue
                article += paragraph.get_text() + '\n'
        
        article = article.strip()
    except Exception as e:
        print('\n--',e,' for the link:',article_url)
        
        # any of the elements cannot be found above return nth
    
    return {'url': article_url,
            'date_publish': date,
            'news_title': news_title,
            'news_category': category,
            'article': article}

out = getNewsByCSS("https://www.channelnewsasia.com/singapore/the-big-read-substation-power-station-arts-artists-363776")

SINGAPORE: Along Armenian Street lies a nondescript grey and white colonial building, which used to be a power station between the late-1920s and 1970s. 
The 1,630-sqm three-storey centre, which was converted from a disused power station, houses a black box theatre, studios and an art gallery. 
However, after about three decades of being a mainstay in the arts scene, The Substation will vacate its premise in July, with the National Arts Council (NAC) taking back the building for renovation works. 
The NAC had said that the building is a conserved property and there is a need for a major upgrading to preserve its structural integrity. The Substation is considering some alternative venues, including Goodman Arts Centre in Mountbatten and Aliwal Arts Centre in Kampong Glam but has yet to finalise its decision. 
From punk rockers to filmmakers, The Substation was a melting pot that allowed artists of different genres to seek out collaborators and make lifelong friends.  
“There was a hole 

In [18]:
# st
#url_1 = 'https://www.straitstimes.com/singapore/singapore-ushers-in-2015-with-brilliant-fireworks-display-song-performances'
#url_2 = 'https://www.straitstimes.com/singapore/what-makes-you-delicious-to-mozzies' #different html layout
#cna
#url_1 = "https://www.channelnewsasia.com/singapore/covid19-coronavirus-measures-vaccination-omicron-dale-fisher-2466416"
#8world
url_1 = "https://www.8world.com/singapore/national-service-registration-1618321"

#zb
url_1 = "https://www.zaobao.com.sg/realtime/singapore/story20220101-1228418"


In [19]:
#8world
title = 'header[class*="article-header"]'

date = 'div[class*="field--type-datetime"]'

category = 'ul>li[class*="article-header"]'

paragraphs = 'div[class"text"] > div[class="text-long"]'

#ZB news
css_title = 'div[class*="article-title"]'

css_date = 'div[class*="article-byline"]'

css_category = 'ul[class*="breadcrumbs-list"]'

css_paragraphs = 'div[class*="article-content-rawhtml"]'


print(scrape_news_By_CSS_Selector(url_1,css_title,css_date,css_category,css_paragraphs))

挥别2021年 10个邻里烟火秀照亮新的一年
{'url': 'https://www.zaobao.com.sg/realtime/singapore/story20220101-1228418', 'date_publish': '文 / \n邓玮婷\n\n\n        \n                  发布 / 2022年1月1日 12:48 AM\n      \n  \n    \n\n        \n                  更新 / 2022年1月1日 12:59 AM', 'news_title': '挥别2021年 10个邻里烟火秀照亮新的一年', 'news_category': '即时\n\n\n\n新加坡', 'article': '（早报讯）细雨中挥别2021年，烟火光下再起航，新年快乐！\n午夜零时一到，夜空飘下细雨，礼花弹往天上冲，水中有火，夜中有光，一场视觉飨宴下为2022年开启新篇章。\n滨海湾跨年倒数活动连续第二年不燃放烟花，不过有10个邻里举办烟花表演，让公众通过线上直播观赏。\n住在联邦一带的周欣妮（40岁，客服人员）今晚特地带七岁女儿下楼看烟花秀跨年。她受访时说，住在这里超过10年，第一次下楼就能看烟花秀。\n“烟花就在眼前，心情很兴奋，烟花秀很精彩，真的没想到能这么近距离看到烟花！老公没法和我们一起看烟花，替他可惜了。”\n“新的一年祝愿大家健康快乐，希望2022年有机会出国游玩。”\n由于新加坡以谨慎方式恢复社交活动，逐步迈向与冠病共存，人民协会连续第二年把社区跨年倒数活动搬上网，烟花秀也再次走入邻里。10个邻里区包括勿洛、碧山、文礼、波纳维斯达、蔡厝港、后港、三巴旺、淡滨尼、兀兰和景万岸－菜市今晚都呈现了烟花秀，把倒数活动推向高潮。\n在三巴旺看烟花秀的潘银银（30岁，待业）说：“希望看了今晚的跨年烟花，来年会幸运一点。”'}
